## Summary
The purpose of this notebook is to reproduce the D- and I-optimal designs for mixture experiments using the settings found in *Development and Applications of Particle Swarm Optimization for Constructing Optimal Experimental Designs* table 6.8. The notebook provides an implementation of an experiment runner, which can be used to distribute the optimizations of many design scenarios with various settings across multiple processes and threads in a flexible and reusable manner.

## Distributed Environment

10-element Vector{Int64}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11

## Imports

In [2]:
using HDF5
using Dates
using Logging
using IterTools

In [7]:
@everywhere include("model_builder/model_builder.jl")
@everywhere using .ModelBuilder

@everywhere include("model_builder/design_initializer.jl")
@everywhere using .DesignInitializer

@everywhere include("optimization/design_optimizer.jl")
@everywhere using .DesignOptimizer

@everywhere include("optimization/optimality_criteria.jl")
@everywhere using .OptimalityCriteria

@everywhere using LinearAlgebra
@everywhere using Optim
@everywhere using ForwardDiff
@everywhere using Optimization, OptimizationNLopt

      From worker 8:	WARNING: replacing module ModelBuilder.
      From worker 10:	WARNING: replacing module ModelBuilder.
      From worker 7:	WARNING: replacing module ModelBuilder.
      From worker 3:	WARNING: replacing module ModelBuilder.
      From worker 5:	WARNING: replacing module ModelBuilder.
      From worker 9:	WARNING: replacing module ModelBuilder.
      From worker 2:	WARNING: replacing module ModelBuilder.
      From worker 6:	WARNING: replacing module ModelBuilder.
      From worker 11:	WARNING: replacing module ModelBuilder.
      From worker 4:	WARNING: replacing module ModelBuilder.


      From worker 2:	WARNING: replacing module DesignInitializer.
      From worker 4:	WARNING: replacing module DesignInitializer.
      From worker 6:	WARNING: replacing module DesignInitializer.
      From worker 8:	WARNING: replacing module DesignInitializer.
      From worker 10:	WARNING: replacing module DesignInitializer.
      From worker 5:	WARNING: replacing module DesignInitializer.
      From worker 11:	WARNING: replacing module DesignInitializer.
      From worker 3:	WARNING: replacing module DesignInitializer.
      From worker 7:	WARNING: replacing module DesignInitializer.
      From worker 9:	WARNING: replacing module DesignInitializer.
      From worker 8:	WARNING: replacing module DesignOptimizer.
      From worker 10:	WARNING: replacing module DesignOptimizer.
      From worker 2:	WARNING: replacing module DesignOptimizer.
      From worker 4:	WARNING: replacing module DesignOptimizer.
      From worker 9:	WARNING: replacing module DesignOptimizer.
      From worker

      From worker 6:	WARNING: replacing module DesignOptimizer.
      From worker 8:	WARNING: replacing module OptimalityCriteria.
      From worker 3:	WARNING: replacing module OptimalityCriteria.
      From worker 10:	WARNING: replacing module OptimalityCriteria.
      From worker 5:	WARNING: replacing module OptimalityCriteria.
      From worker 9:	WARNING: replacing module OptimalityCriteria.
      From worker 2:	WARNING: replacing module OptimalityCriteria.
      From worker 11:	WARNING: replacing module OptimalityCriteria.
      From worker 7:	WARNING: replacing module OptimalityCriteria.
      From worker 4:	WARNING: replacing module OptimalityCriteria.
      From worker 6:	WARNING: replacing module OptimalityCriteria.


## Table of Design Scenarios
| Criterion   | Model           | K  | N  | Ground Truth                  |
|-------------|-----------------|----|----|-------------------------------|
| D-criterion | first-order     | 3  | 3  | {3, 1}                        | 
| D-criterion | second-order    | 3  | 6  | {3, 2}                        | 
| D-criterion | first-order     | 4  | 4  | {4, 1}                        | 
| D-criterion | second-order    | 4  | 10 | {4, 2}                        | 
| D-criterion | special cubic   | 3  | 7  | {3, 3}-SCD                    | 
| D-criterion | special cubic   | 4  | 14 | {4, 3}-SCD                    | 
| D-criterion | full cubic      | 3  | 10 | augmented {4, 3}-SCD          | 
| D-criterion | full cubic      | 4  | 20 | augmented {4, 4}-SCD          | 
| IV-criterion| second-order    | 3  | 6  | Goos et al. (2016)            | 
| IV-criterion| second-order    | 3  | 7  | Goos et al. (2016)            | 
| IV-criterion| second-order    | 3  | 8  | Goos et al. (2016)            | 
| IV-criterion| second-order    | 3  | 30 | Goos et al. (2016)            | 
| IV-criterion| second-order    | 4  | 15 | Goos et al. (2016)            | 
| IV-criterion| special-cubic   | 4  | 16 | Goos et al. (2016)            | 
| IV-criterion| special-cubic   | 4  | 17 | Goos et al. (2016)            |


## CEXCH Implementation

In [58]:
@everywhere begin
    function basic_score_comparator(a, b)
        a < b
    end
    
    function create_univariate_objective(X, row, d, obj_crit)
        og_row = copy(X[row, :])
        function univariate_obj(x)
            X[row, :] .= og_row .+ x * d
            score = obj_crit(X)
            X[row, :] .= og_row
            return score
        end
        return univariate_obj
    end
    
    function bounded_univariate_optimizer(X, row, d, obj_crit)
        univariate_obj = create_univariate_objective(X, row, d, obj_crit)
        result = Optim.optimize(univariate_obj, 0, 1.0)
        optim_point = X[row, :] .+ Optim.minimizer(result) * d
        return optim_point, Optim.minimum(result)
    end

    function bounded_univariate_gradient_optimizer(X, row, d, obj_crit; optimizer=NLopt.LD_BFGS)
        # Make univariate optimizer
        univariate_obj = create_univariate_objective(X, row, d, obj_crit)
        univariate_obj_vec = (x, _) -> univariate_obj(x[1])

        # Initial values and bounds
        x0 = [0]
        p=[0]
        lb = [0]
        ub = [1.0]

        # Define optimization problem
        f = OptimizationFunction(univariate_obj_vec, Optimization.AutoForwardDiff())
        prob = OptimizationProblem(f, x0, p, ub=ub, lb=lb)
        solution = solve(prob, optimizer())
        optim_point = X[row, :] .+ solution.u[1] * d

        # Evaluate
        tmp = copy(X[row, :])
        X[row, :] .= optim_point
        score = obj_crit(X)
        X[row, :] .= tmp

        return optim_point, score
    end

    function bounded_univariate_gradient_optimizer(optimizer=NLopt.LD_BFGS)
        return (X, row, d, obj_crit) -> bounded_univariate_gradient_optimizer(X, row, d, obj_crit; optimizer=optimizer)
    end
    
    function cexch_optimize(
            X_in::Matrix{Float64}, 
            obj_crit::Function, 
            optimizer::Function; 
            max_num_iters=1000, 
            score_comparator=basic_score_comparator
        )

        # Copy the input design matrix
        X = copy(X_in)
    
        # Get N and K
        N, K = size(X)
    
        # Generate simplex coordinates using identity matrix
        # The kth column corresponds with the simplex vertex for the kth factor in an unconstrained setting
        simplex_coords = I(K)
    
        # Initialize objective value
        best_score = obj_crit(X)

        # Iterate until no improvement is made
        iter = 0
        while iter < max_num_iters
            improvement = false
    
            for coord in CartesianIndices(X)
                row, col = coord[1], coord[2]
    
                # Get the current simplex vertex
                v = simplex_coords[:, col]
    
                # Get the direction vector
                d = v - X[row, :]
    
                # Optimize the objective function along the direction vector
                optim_point, score_opt = optimizer(X, row, d, obj_crit)
    
                # Update the design matrix and objective value if improvement is found
                if score_comparator(score_opt, best_score)
                    best_score = score_opt
                    X[row, :] .= optim_point
                    improvement = true
                end
            end
    
            if !improvement
                break
            end

            iter += 1
        end
    
        return X, best_score, iter
    end
    
    function cexch(
            X::Array{Float64, 3},
            obj_crit::Function, 
            optimizer::Function; 
            max_iters=1000, 
            score_comparator=basic_score_comparator
        )
        
        # Get the number of initial design matrices
        n, N, K = size(X) # number of initial design matrices

        # Initialize arrays to store scores and number of iterations    
        scores = Float64[]
        num_iterations = Int[]
        opt_designs = zeros(n, N, K)

        sizeX = size(X)
        sizeOpt = size(opt_designs)
    
        # Parallelize over initializations
        for i in 1:n
            optimized, best_score, num_iters = cexch_optimize(X[i, :, :], obj_crit, optimizer; max_num_iters=max_iters, score_comparator=score_comparator)
            push!(scores, best_score)
            push!(num_iterations, num_iters)
            opt_designs[i, :, :] .= optimized
        end
    
        # return scores, num_iterations    
        return opt_designs, scores, num_iterations
    end
end

## Experiment Runner
The experiment runner will distribute the experiments across multiple processes. Each experimental design scenario will run inside a process using parallelization.

### Types

In [59]:
@everywhere begin 
    struct DesignOptimizationScenario
        model_builder::Function
        K::Int
        N::Int
        max_iters::Int
        num_initializations::Int
    end
    
    struct DesignOptimizationJob
        scenarios::Vector{DesignOptimizationScenario}
        design_initializer::Function
        optimizer::Function
        optimality_criterion::Function
    end

    struct JobParams
        N::Int
        K::Int
        model_builder::Function
        design_initializer::Function
        optimizer::Function
        optimality_criterion::Function
        max_iters::Int
        num_initializations::Int
    end
    
    struct DesignOptimizationResult
        optimizers::Array
        optima::Vector
        iterations::Vector
    end
    
    function create_scenario(model_builder, K, N; max_iters::Integer = 100, num_initializations::Integer = 50)
        return DesignOptimizationScenario(model_builder, K, N, max_iters, num_initializations)
    end
    
    function create_result(X, optima, iterations)
        return DesignOptimizationResult(X, optima, iterations)
    end
end

### Batch and Distribute Jobs

In [60]:
@everywhere function optimize_designs(params::JobParams)::DesignOptimizationResult
    @info "Starting job for N=$(params.N), K=$(params.K), model=$(params.model_builder)"

    # Initialize designs
    init = params.design_initializer(params.N, params.K, params.model_builder)
    initial_designs = init(params.num_initializations)

    # Create objective criterion
    obj_crit = params.optimality_criterion ∘ params.model_builder

    # Run optimization
    X, scores, num_iterations = cexch(initial_designs, obj_crit, params.optimizer; max_iters=params.max_iters)

    @info "Starting job for N=$(params.N), K=$(params.K), model=$(params.model_builder)"

    return create_result(X, scores, num_iterations)
end

In [3]:
arr = rand(3,4,5)
size(arr)

(3, 4, 5)

In [5]:
arr[1:3, :, :]

3×4×5 Array{Float64, 3}:
[:, :, 1] =
 0.721751  0.589597  0.44613   0.638989
 0.128607  0.447186  0.663277  0.165968
 0.1803    0.554536  0.639503  0.404908

[:, :, 2] =
 0.26761   0.72824   0.711621  0.486031
 0.230464  0.363134  0.228169  0.92967
 0.279201  0.34153   0.253589  0.511162

[:, :, 3] =
 0.165116   0.661902   0.75092   0.38638
 0.956754   0.914163   0.877312  0.44696
 0.0838368  0.0265921  0.341125  0.050911

[:, :, 4] =
 0.386643  0.715479  0.402969  0.783423
 0.557022  0.527692  0.495767  0.645896
 0.865549  0.233818  0.14035   0.990834

[:, :, 5] =
 0.1489    0.650166  0.869008  0.66373
 0.641994  0.34469   0.773553  0.236572
 0.176202  0.934787  0.2878    0.544205

In [61]:
function batch_upper_indices(vector_length, batch_size)
    indices = [min(i + batch_size - 1, vector_length) for i in 1:batch_size:vector_length]
    return [0; indices]
end

function batch_scenarios(job::DesignOptimizationJob; batch_size::Integer = 50)::Vector{JobParams}
    all_jobs = []
    for scenario in job.scenarios
        batch_indices = batch_upper_indices(scenario.num_initializations, batch_size)
        for i in 2:size(batch_indices, 1)
            push!(all_jobs, JobParams(
                scenario.N,
                scenario.K, 
                scenario.model_builder, 
                job.design_initializer, 
                job.optimizer, 
                job.optimality_criterion,
                scenario.max_iters,
                (batch_indices[i] - batch_indices[i - 1])
            ))
        end
    end

    return all_jobs
end

batch_scenarios (generic function with 1 method)

In [62]:
function run_optimization_job(job::DesignOptimizationJob; batch_size::Integer = 50)::Vector{DesignOptimizationResult}
    all_jobs = batch_scenarios(job, batch_size=batch_size)   
    pmap(optimize_designs, all_jobs)
end

run_optimization_job (generic function with 1 method)

## Define Scenarios

In [63]:
scenarios = [
    create_scenario(ModelBuilder.scheffe(1), 3, 3),
    # create_scenario(ModelBuilder.scheffe(2), 3, 6),
    # create_scenario(ModelBuilder.scheffe(1), 4, 4),
    # create_scenario(ModelBuilder.scheffe(2), 4, 10),
    # create_scenario(ModelBuilder.special_cubic, 3, 7),
    # create_scenario(ModelBuilder.special_cubic, 4, 14),
    # create_scenario(ModelBuilder.full_cubic, 3, 10),
    # create_scenario(ModelBuilder.full_cubic, 4, 20)
]

mixture_initializer = (n, k, builder) -> DesignInitializer.initializer(n, k, builder, type="mixture")

optimization_job = DesignOptimizationJob(
    scenarios,
    mixture_initializer,
    bounded_univariate_gradient_optimizer(NLopt.LD_LBFGS),
    OptimalityCriteria.d_criterion
)

DesignOptimizationJob(DesignOptimizationScenario[DesignOptimizationScenario(Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}(1, 0, false, Any[], false, true), 3, 3, 100, 50)], var"#144#145"(), var"#133#138"{Algorithm}(NLopt.LD_LBFGS), Main.OptimalityCriteria.d_criterion)

In [64]:
results = run_optimization_job(optimization_job; batch_size=50)

      From worker 8:	[ Info: Starting job for N=3, K=3, model=model_builder
      From worker 8:	┌ Warning: NLopt failed to converge: FORCED_STOP
      From worker 8:	└ @ OptimizationNLopt C:\Users\benja\.julia\packages\OptimizationNLopt\CgKow\src\OptimizationNLopt.jl:181
      From worker 8:	[ Info: [0.0]


RemoteException: On worker 8:
MethodError: no method matching *(::SciMLBase.OptimizationSolution{Float64, 1, Vector{Float64}, OptimizationCache{OptimizationFunction{true, AutoForwardDiff{nothing, Nothing}, Serialization.__deserialized_types__.var"#82#87"{Serialization.__deserialized_types__.var"#univariate_obj#86"{Matrix{Float64}, Int64, Vector{Float64}, ComposedFunction{typeof(Main.OptimalityCriteria.d_criterion), Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool, Bool}}, Vector{Float64}}}, OptimizationForwardDiffExt.var"#38#56"{ForwardDiff.GradientConfig{ForwardDiff.Tag{OptimizationForwardDiffExt.var"#37#55"{OptimizationFunction{true, AutoForwardDiff{nothing, Nothing}, Serialization.__deserialized_types__.var"#82#87"{Serialization.__deserialized_types__.var"#univariate_obj#86"{Matrix{Float64}, Int64, Vector{Float64}, ComposedFunction{typeof(Main.OptimalityCriteria.d_criterion), Main.ModelBuilder.var"#model_builder#19"{Int64, Int64, Bool, Vector{Any}, Bool